# TASK #1

In [1]:
import websocket
import json
import threading

#########
PREDEFINED_TICKERS = ["BTC-USD", "ETH-USD"]


current_data = {ticker: {} for ticker in PREDEFINED_TICKERS}

def print_ticker_info(ticker):
    ticker_data = current_data[ticker]
    formatted_info = f"""
    Ticker: {ticker}
    Funding Rate: {ticker_data.get('nextFundingRate', 'N/A')}
    Open Interest: {ticker_data.get('openInterest', 'N/A')}
    Oracle Price: {ticker_data.get('oraclePrice', 'N/A')}
    24h Volume: {ticker_data.get('volume24H', 'N/A')}
    24h Trades: {ticker_data.get('trades24H', 'N/A')}
    Next Funding: {ticker_data.get('nextFundingRate', 'N/A')}
    ---------------------------------------------
    """
    print(formatted_info)

def update_ticker_info(ticker, data):
    current_data[ticker].update(data)

def on_message(ws, message):
    data = json.loads(message)
    
    if data.get('type') == 'subscribed' and 'markets' in data.get('contents', {}):
        markets_data = data['contents']['markets']
        for ticker in PREDEFINED_TICKERS:
            market_data = markets_data.get(ticker)
            if market_data:
                update_ticker_info(ticker, market_data)
                print_ticker_info(ticker)

    elif data.get('type') == 'channel_data' and 'trading' in data.get('contents', {}):
        trading_data = data['contents']['trading']
        for ticker in PREDEFINED_TICKERS:
            if ticker in trading_data:
                update_ticker_info(ticker, trading_data[ticker])
                print_ticker_info(ticker)

def on_error(ws, error):
    print("Error:", error)

def on_close(ws, close_status_code, close_msg):
    print("### CLOSED CONNECTION ###")

def on_open(ws):
    def run(*args):
    
        subscription_message = json.dumps({
            "type": "subscribe",
            "channel": "v4_markets",
        })
        ws.send(subscription_message)
        print("Market channel subscription: v4_markets.")

    threading.Thread(target=run).start()

websocket_url = "wss://indexer.v4testnet.dydx.exchange/v4/ws"
ws = websocket.WebSocketApp(websocket_url, on_message=on_message, on_error=on_error, on_close=on_close)
ws.on_open = on_open
ws.run_forever()


Market channel subscription: v4_markets.

    Ticker: BTC-USD
    Funding Rate: 0
    Open Interest: 106.1114
    Oracle Price: 42858.88341
    24h Volume: 422735.0415
    24h Trades: 6538
    Next Funding: 0
    ---------------------------------------------
    

    Ticker: ETH-USD
    Funding Rate: 0
    Open Interest: 1407.402
    Oracle Price: 2321.842885
    24h Volume: 381509.2673
    24h Trades: 6678
    Next Funding: 0
    ---------------------------------------------
    

    Ticker: BTC-USD
    Funding Rate: 0
    Open Interest: 106.1114
    Oracle Price: 42858.88341
    24h Volume: 422663.3401
    24h Trades: 6538
    Next Funding: 0
    ---------------------------------------------
    

    Ticker: ETH-USD
    Funding Rate: 0
    Open Interest: 1407.430
    Oracle Price: 2321.842885
    24h Volume: 381500.1161
    24h Trades: 6679
    Next Funding: 0
    ---------------------------------------------
    
Error: 
### CLOSED CONNECTION ###


True

# TASK #2

In [2]:
import websocket
import json
import threading
from v4_client_py import IndexerClient
from v4_client_py.clients.constants import Network

####################
TICKERS = ["BTC-USD", "ETH-USD"]
####################


 
client = IndexerClient(
    config=Network.testnet().indexer_config,
)


def print_orderbook_info(ticker, data):
    time_response = client.utility.get_time()
    timestamp = time_response.data['iso']
    #timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"{ticker}: ({timestamp})")
    if 'bids' in data and data['bids']:
        print(f"  Bid: {data['bids'][0][0]} | Bid Size: {data['bids'][0][1]}")
    if 'asks' in data and data['asks']:
        print(f"  Ask: {data['asks'][0][0]} | Ask Size: {data['asks'][0][1]}")
    print("---------------------------------------------")

def on_message(ws, message):
    data = json.loads(message)
    

    if data.get('type') == 'channel_data' and 'contents' in data:
        contents = data['contents']
        if 'bids' in contents or 'asks' in contents:
            ticker = data['id']
            print_orderbook_info(ticker, contents)

def on_error(ws, error):
    print("Error:", error)

def on_close(ws, close_status_code, close_msg):
    print("### Connection closed ###")

def on_open(ws):
    def run(*args):
        for ticker in TICKERS:
   
            subscription_message = json.dumps({
                "type": "subscribe",
                "channel": "v4_orderbook",
                "id": ticker
            })
            ws.send(subscription_message)
            print(f"{ticker} Orderbook v4 subscription sended.")

    threading.Thread(target=run).start()


websocket_url = "wss://indexer.v4testnet.dydx.exchange/v4/ws"
ws = websocket.WebSocketApp(websocket_url, on_message=on_message, on_error=on_error, on_close=on_close)
ws.on_open = on_open
ws.run_forever()

BTC-USD Orderbook v4 subscription sended.
ETH-USD Orderbook v4 subscription sended.
BTC-USD: (2024-02-06T03:21:07.798Z)
  Ask: 43102 | Ask Size: 0.0015
---------------------------------------------
BTC-USD: (2024-02-06T03:21:08.522Z)
  Ask: 43145 | Ask Size: 0.0015
---------------------------------------------
BTC-USD: (2024-02-06T03:21:09.238Z)
  Bid: 42887 | Bid Size: 0.0015
---------------------------------------------
BTC-USD: (2024-02-06T03:21:09.897Z)
  Ask: 42973 | Ask Size: 0.0015
---------------------------------------------
BTC-USD: (2024-02-06T03:21:10.556Z)
  Ask: 43016 | Ask Size: 0.0015
---------------------------------------------
BTC-USD: (2024-02-06T03:21:11.221Z)
  Ask: 42926 | Ask Size: 0
---------------------------------------------
BTC-USD: (2024-02-06T03:21:11.875Z)
  Ask: 42919 | Ask Size: 0
---------------------------------------------
BTC-USD: (2024-02-06T03:21:12.544Z)
  Ask: 43059 | Ask Size: 0.0015
---------------------------------------------
BTC-USD: (2024

True

# TASK #3

In [3]:
from v4_client_py import IndexerClient
from v4_client_py.clients.constants import Network, BECH32_PREFIX
from v4_client_py.chain.aerial.wallet import LocalWallet#, BECH32_PREFIX
import requests


headers = {
  'Accept': 'application/json'
}


 #### MODIFY TO THE CORRECT ADDRESS
r = requests.get('https://dydx-testnet.imperator.co/v4/addresses/[AccountAddress]', headers = headers)
r= r.json()


client = IndexerClient(
    config=Network.testnet().indexer_config,
)

 #### MODIFY TO THE CORRECT ADDRESS
# address is the wallet address on dYdX chain, subaccount_number is the subaccount number
address = 'usethecorrectaddress'
subaccounts_response = client.account.get_subaccounts(address)
subaccounts = subaccounts_response.data['subaccounts']


# SUBACCOUNT NUMBER

subaccount_number = 0
######################


perpetual_positions_response = client.account.get_subaccount_perpetual_positions(address, subaccount_number)
perpetual_positions = perpetual_positions_response.data['positions']

def print_positions(positions):
    for pos in positions:
        ticker = pos['market']
        side = pos['side']
        size = pos['size']
        leverage = "N/A"  # No presente en la documentacion 
        liquidation_price = "N/A"  # No presente en la documentacion 
        unrealized_pnl = pos['unrealizedPnl']
        realized_pnl = pos['realizedPnl']
        margin_usage = "N/A"  # No presente en la documentacion 
        free_collateral = r['subaccounts'][0]['freeCollateral']  
        avg_open_close = pos['sumOpen'] + '/' + pos['sumClose']  
        
        print(f"Ticker : {ticker}\nSide : {side}\nPosition Size : {size}\nLeverage : {leverage}\nLiquidation Price : {liquidation_price}\nUnrealized PnL : {unrealized_pnl}\nRealized PnL : {realized_pnl}\nMargin Usage : {margin_usage}\nFree Collateral : {free_collateral}\nsumOpen / sumClose : {avg_open_close}")


print_positions(perpetual_positions)


Ticker : ETH-USD
Side : LONG
Position Size : 0.022
Leverage : N/A
Liquidation Price : N/A
Unrealized PnL : 0.669332664
Realized PnL : 0
Margin Usage : N/A
Free Collateral : 198.0928980308
sumOpen / sumClose : 0.022/0
